In [1]:
# https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html
# https://colab.research.google.com/drive/1uFJBO1pgsiFwCGIJwZlhUzaJ2srDbtw-#scrollTo=LjMMYJv85hVT
# https://www.tensorflow.org/tutorials/text/nmt_with_attention

In [1]:
import pandas as pd
import numpy as np
import torch
import torchvision
import torch.functional as F
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader,TensorDataset
from sklearn.model_selection import train_test_split
import os
import unicodedata
import re
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import time

In [2]:
def load_data(loc = '../input/spa-eng/spa.txt', num_samples = 30000 ):
    file = open(loc, encoding='UTF-8').read().strip().split('\n')  
    original_word_pairs = [[w for w in l.split('\t')] for l in file[:num_samples]]
    data = pd.DataFrame(original_word_pairs, columns=["eng", "es"])
    return data

# Converts the unicode file to ascii
def unicode_to_ascii(s):
    """
    Normalizes latin chars with accent to their canonical decomposition
    """
    return ''.join(c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn')

def preprocess_sentence(w):
    w = unicode_to_ascii(w.lower().strip())
    
    # creating a space between a word and the punctuation following it
    # eg: "he is a boy." => "he is a boy ." 
    # Reference:- https://stackoverflow.com/questions/3645931/python-padding-punctuation-with-white-spaces-keeping-punctuation
    w = re.sub(r"([?.!,¿])", r" \1 ", w)
    w = re.sub(r'[" "]+', " ", w)
    
    # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")
    w = re.sub(r"[^a-zA-Z?.!,¿]+", " ", w)
    w = w.rstrip().strip()
    
    # adding a start and an end token to the sentence
    # so that the model know when to start and stop predicting.
    w = '<start> ' + w + ' <end>'
    return w
    

In [3]:
class LanguageIndex():
    def __init__(self, lang):
        """ lang are the list of phrases from each language"""
        self.lang = lang
        self.word2idx = {}
        self.idx2word = {}
        self.vocab = set()
        
        self.create_index()
        
    def create_index(self):
        for phrase in self.lang:
            # update with individual tokens
            self.vocab.update(phrase.split(' '))
            
        # sort the vocab
        self.vocab = sorted(self.vocab)

        # add a padding token with index 0
        self.word2idx['<pad>'] = 0
        
        # word to index mapping
        for index, word in enumerate(self.vocab):
            self.word2idx[word] = index + 1 # +1 because of pad token
        
        # index to word mapping
        for word, index in self.word2idx.items():
            self.idx2word[index] = word        

In [4]:
def max_length(tensor):
    return max(len(t) for t in tensor)

In [5]:
data = load_data()
data.head(10)

,eng,es
0,Go.,Ve.
1,Go.,Vete.
2,Go.,Vaya.
3,Go.,Váyase.
4,Hi.,Hola.
5,Run!,¡Corre!
6,Run.,Corred.
7,Who?,¿Quién?
8,Fire!,¡Fuego!
9,Fire!,¡Incendio!


In [6]:
# Now we do the preprocessing using pandas and lambdas
data["eng"] = data.eng.apply(lambda w: preprocess_sentence(w))
data["es"] = data.es.apply(lambda w: preprocess_sentence(w))
data.sample(10)

,eng,es
29607,<start> tom has three cousins . <end>,<start> tom tiene tres primos . <end>
3257,<start> tell us more . <end>,<start> cuentanos mas . <end>
11641,<start> the book is here . <end>,<start> el libro esta aqui . <end>
13938,<start> it s dark outside . <end>,<start> esta oscuro fuera . <end>
12821,<start> do you have a car ? <end>,<start> ¿ tienes auto ? <end>
8565,<start> it s not so far . <end>,<start> no esta tan lejos . <end>
16272,<start> i already said yes . <end>,<start> ya dije que si . <end>
16472,<start> i have cabin fever . <end>,<start> tengo miedo a los lugares cerrados . <...
19393,<start> don t ever doubt it . <end>,<start> ni lo dudes . <end>
23757,<start> i drank from the tap . <end>,<start> he bebido de la canilla . <end>


In [7]:
# index language using the class above
inp_lang = LanguageIndex(data["es"].values.tolist())
targ_lang = LanguageIndex(data["eng"].values.tolist())
# Vectorize the input and target languages
input_tensor = [[inp_lang.word2idx[s] for s in es.split(' ')]  for es in data["es"].values.tolist()]
target_tensor = [[targ_lang.word2idx[s] for s in eng.split(' ')]  for eng in data["eng"].values.tolist()]
input_tensor[:10]

[[5, 9090, 3, 4],
 [5, 9204, 3, 4],
 [5, 9082, 3, 4],
 [5, 9089, 3, 4],
 [5, 4702, 3, 4],
 [5, 2299, 1, 4],
 [5, 2304, 3, 4],
 [5, 9413, 7433, 6, 4],
 [5, 4270, 1, 4],
 [5, 4881, 1, 4]]

In [8]:
target_tensor[:10]

[[5, 1857, 3, 4],
 [5, 1857, 3, 4],
 [5, 1857, 3, 4],
 [5, 1857, 3, 4],
 [5, 2058, 3, 4],
 [5, 3655, 1, 4],
 [5, 3655, 3, 4],
 [5, 4815, 6, 4],
 [5, 1636, 1, 4],
 [5, 1636, 1, 4]]

In [9]:
# calculate the max_length of input and output tensor
max_length_inp, max_length_tar = max_length(input_tensor), max_length(target_tensor)

In [11]:
max_length_tar

11

In [13]:
def pad_sequences(x, max_len):
    padded = np.zeros((max_len), dtype=np.int64)
    if len(x) > max_len: padded[:] = x[:max_len]
    else: padded[:len(x)] = x
    return padded

In [14]:
# inplace padding
input_tensor = [pad_sequences(x, max_length_inp) for x in input_tensor]
target_tensor = [pad_sequences(x, max_length_tar) for x in target_tensor]
len(target_tensor)

30000

In [17]:
target_tensor[0]

array([   5, 1857,    3,    4,    0,    0,    0,    0,    0,    0,    0],
      dtype=int64)

In [18]:
print("max_length_inp :", max_length_inp)
print("max_length_tar :",max_length_tar)

max_length_inp : 16
max_length_tar : 11


In [19]:
# Creating training and validation sets using an 80-20 split
input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(input_tensor, target_tensor, test_size=0.2)

# Show length
len(input_tensor_train), len(target_tensor_train), len(input_tensor_val), len(target_tensor_val)

(24000, 24000, 6000, 6000)

In [25]:
class MyData(Dataset):
    def __init__(self, X, y):
        self.data = X
        self.target = y
        self.length = [ np.sum(1 - np.equal(x, 0)) for x in X]
        
    def __getitem__(self, index):
        x = self.data[index]
        y = self.target[index]
        x_len = self.length[index]
        return x,y,x_len
    
    def __len__(self):
        return len(self.data)

In [26]:
train_dat = MyData(input_tensor_train, target_tensor_train)

In [27]:
BUFFER_SIZE = len(input_tensor_train)
BATCH_SIZE = 64
N_BATCH = BUFFER_SIZE//BATCH_SIZE
embedding_dim = 256
units = 1024
vocab_inp_size = len(inp_lang.word2idx)
vocab_tar_size = len(targ_lang.word2idx)

In [28]:
vocab_tar_size

4935

In [29]:
dat = DataLoader(train_dat, batch_size = BATCH_SIZE, 
                     drop_last=True,
                     shuffle=True)
it = iter(dat)
x, y, x_len = next(it)

In [30]:
x[1]

tensor([   5, 4618,  360,    3,    4,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0])

In [33]:
x_len

tensor([ 6,  5,  7,  7,  8,  5,  6,  8,  6,  8,  6,  6,  6,  8,  7,  6, 10,  6,
         5,  6,  6,  7,  7,  8,  7,  8,  6,  7,  8,  6,  7,  5,  8,  8,  5,  5,
         5,  7,  7,  7,  7,  7,  7,  7,  7, 10,  6,  6,  7,  7,  5,  9,  9,  7,
         7,  7,  6,  5,  7,  8,  6,  8,  7,  7], dtype=torch.int32)

In [34]:
train_dataset = MyData(input_tensor_train, target_tensor_train)
val_dataset = MyData(input_tensor_val, target_tensor_val)

dataset = DataLoader(train_dataset, batch_size = BATCH_SIZE, 
                     drop_last=True,
                     shuffle=True)

In [35]:
BUFFER_SIZE = len(input_tensor_train)
BATCH_SIZE = 64
N_BATCH = BUFFER_SIZE//BATCH_SIZE
embedding_dim = 256
units = 1024
vocab_inp_size = len(inp_lang.word2idx)
vocab_tar_size = len(targ_lang.word2idx)

In [36]:
dataset = DataLoader(train_dataset, batch_size = BATCH_SIZE, 
                     drop_last=True,
                     shuffle=True)

In [37]:
class Encoder(nn.Module):
    def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
        super(Encoder, self).__init__()
        self.batch_sz = batch_sz
        self.enc_units = enc_units
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.embedding = nn.Embedding(self.vocab_size, self.embedding_dim)
        self.gru = nn.GRU(self.embedding_dim, self.enc_units)
        
    def forward(self, x, lens, device):
        # x: batch_size, max_length 
        
        # x: batch_size, max_length, embedding_dim
        x = self.embedding(x) 
#         print(1,x.shape)
                
        # x transformed = max_len X batch_size X embedding_dim
        # x = x.permute(1,0,2)
        x = pack_padded_sequence(x, lens) # unpad
#         print(2,x)
    
        self.hidden = self.initialize_hidden_state(device)
#         print(3, self.hidden.shape)
        
        # output: max_length, batch_size, enc_units
        # self.hidden: 1, batch_size, enc_units
        output, self.hidden = self.gru(x, self.hidden) # gru returns hidden state of all timesteps as well as hidden state at last timestep
#         print(4,output, self.hidden.shape)
        # pad the sequence to the max length in the batch
        output, _ = pad_packed_sequence(output)
#         print(5, output.shape)
        
        return output, self.hidden

    def initialize_hidden_state(self, device):
        return torch.zeros((1, self.batch_sz, self.enc_units)).to(device)

In [38]:
### sort batch function to be able to use with pad_packed_sequence
def sort_batch(X, y, lengths):
    lengths, indx = lengths.sort(dim=0, descending=True)
    X = X[indx]
    y = y[indx]
    return X.transpose(0,1), y, lengths # transpose (batch x seq) to (seq x batch)

In [39]:
### Testing Encoder part
# TODO: put whether GPU is available or not
# Device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)

encoder.to(device)
# obtain one sample from the data iterator
it = iter(dataset)
x, y, x_len = next(it)

# sort the batch first to be able to use with pac_pack_sequence
xs, ys, lens = sort_batch(x, y, x_len)

enc_output, enc_hidden = encoder(xs.to(device), lens, device)

print(enc_output.size()) # max_length, batch_size, enc_units

torch.Size([10, 64, 1024])


In [40]:
class Decoder(nn.Module):
    def __init__(self, vocab_size, embedding_dim, dec_units, enc_units, batch_sz):
        super(Decoder, self).__init__()
        self.batch_sz = batch_sz
        self.dec_units = dec_units
        self.enc_units = enc_units
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.embedding = nn.Embedding(self.vocab_size, self.embedding_dim)
        self.gru = nn.GRU(self.embedding_dim + self.enc_units, 
                          self.dec_units,
                          batch_first=True)
        self.fc = nn.Linear(self.enc_units, self.vocab_size)
        
        # used for attention
        self.W1 = nn.Linear(self.enc_units, self.dec_units)
        self.W2 = nn.Linear(self.enc_units, self.dec_units)
        self.V = nn.Linear(self.enc_units, 1)
    
    def forward(self, x, hidden, enc_output):
        # enc_output original: (max_length, batch_size, enc_units)
        # enc_output converted == (batch_size, max_length, hidden_size)
        enc_output = enc_output.permute(1,0,2)
        print(1,enc_output.shape)
        # hidden shape == (batch_size, hidden size)
        # hidden_with_time_axis shape == (batch_size, 1, hidden size)
        # we are doing this to perform addition to calculate the score
        
        # hidden shape == (batch_size, hidden size)
        # hidden_with_time_axis shape == (batch_size, 1, hidden size)
        hidden_with_time_axis = hidden.permute(1, 0, 2)
        
        # score: (batch_size, max_length, hidden_size) # Bahdanaus's
        # we get 1 at the last axis because we are applying tanh(FC(EO) + FC(H)) to self.V
        # It doesn't matter which FC we pick for each of the inputs
        score = torch.tanh(self.W1(enc_output) + self.W2(hidden_with_time_axis))
        
        #score = torch.tanh(self.W2(hidden_with_time_axis) + self.W1(enc_output))
          
        # attention_weights shape == (batch_size, max_length, 1)
        # we get 1 at the last axis because we are applying score to self.V
        attention_weights = torch.softmax(self.V(score), dim=1)
        print(2, attention_weights.shape)
        
        # context_vector shape after sum == (batch_size, hidden_size)
        context_vector = attention_weights * enc_output
        context_vector = torch.sum(context_vector, dim=1)
        print(3, context_vector.shape)
        
        # x shape after passing through embedding == (batch_size, 1, embedding_dim)
        # takes case of the right portion of the model above (illustrated in red)
        x = self.embedding(x)
        print(4, x.shape)
        
        # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)
        #x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)
        # ? Looks like attention vector in diagram of source
        x = torch.cat((context_vector.unsqueeze(1), x), -1)
        print(5, x.shape)
        
        # passing the concatenated vector to the GRU
        # output: (batch_size, 1, hidden_size)
        output, state = self.gru(x)
        print(6,output.shape,state.shape)
        
        
        # output shape == (batch_size * 1, hidden_size)
        output =  output.view(-1, output.size(2))
        print(7, output.shape)
        
        # output shape == (batch_size * 1, vocab)
        x = self.fc(output)
        print(x, x.shape)
        
        return x, state, attention_weights
    
    def initialize_hidden_state(self):
        return torch.zeros((1, self.batch_sz, self.dec_units))

In [41]:
# Device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)

encoder.to(device)
# obtain one sample from the data iterator
it = iter(dataset)
x, y, x_len = next(it)

print("Input: ", x.shape)
print("Output: ", y.shape)

# sort the batch first to be able to use with pac_pack_sequence
xs, ys, lens = sort_batch(x, y, x_len)

enc_output, enc_hidden = encoder(xs.to(device), lens, device)
print("Encoder Output: ", enc_output.shape) # batch_size X max_length X enc_units
print("Encoder Hidden: ", enc_hidden.shape) # batch_size X enc_units (corresponds to the last state)

decoder = Decoder(vocab_tar_size, embedding_dim, units, units, BATCH_SIZE)
decoder = decoder.to(device)

#print(enc_hidden.squeeze(0).shape)

dec_hidden = enc_hidden#.squeeze(0)
dec_input = torch.tensor([[targ_lang.word2idx['<start>']]] * BATCH_SIZE)
print("Decoder Input: ", dec_input.shape)
print("--------")

for t in range(1, y.size(1)):
    # enc_hidden: 1, batch_size, enc_units
    # output: max_length, batch_size, enc_units
    predictions, dec_hidden, _ = decoder(dec_input.to(device), 
                                         dec_hidden.to(device), 
                                         enc_output.to(device))
    
    print("Prediction: ", predictions.shape)
    print("Decoder Hidden: ", dec_hidden.shape)
    
    #loss += loss_function(y[:, t].to(device), predictions.to(device))
    
    dec_input = y[:, t].unsqueeze(1)
    print(dec_input.shape)
    break

Input:  torch.Size([64, 16])
Output:  torch.Size([64, 11])
Encoder Output:  torch.Size([10, 64, 1024])
Encoder Hidden:  torch.Size([1, 64, 1024])
Decoder Input:  torch.Size([64, 1])
--------
1 torch.Size([64, 10, 1024])
2 torch.Size([64, 10, 1])
3 torch.Size([64, 1024])
4 torch.Size([64, 1, 256])
5 torch.Size([64, 1, 1280])
6 torch.Size([64, 1, 1024]) torch.Size([1, 64, 1024])
7 torch.Size([64, 1024])
tensor([[-0.0074, -0.0294, -0.0218,  ...,  0.0132, -0.0486,  0.0067],
        [-0.0263, -0.0302, -0.0016,  ...,  0.0133, -0.0428,  0.0211],
        [-0.0052, -0.0052, -0.0109,  ...,  0.0143, -0.0330, -0.0091],
        ...,
        [-0.0171, -0.0297, -0.0117,  ...,  0.0043, -0.0456, -0.0051],
        [-0.0251, -0.0298, -0.0070,  ...,  0.0181, -0.0365, -0.0006],
        [-0.0305, -0.0214,  0.0003,  ...,  0.0219, -0.0343,  0.0002]],
       device='cuda:0', grad_fn=<AddmmBackward>) torch.Size([64, 4935])
Prediction:  torch.Size([64, 4935])
Decoder Hidden:  torch.Size([1, 64, 1024])
torch.Size

In [41]:
criterion = nn.CrossEntropyLoss()

def loss_function(real, pred):
    """ Only consider non-zero inputs in the loss; mask needed """
    #mask = 1 - np.equal(real, 0) # assign 0 to all above 0 and 1 to all 0s
    #print(mask)
    mask = real.ge(1).type(torch.cuda.FloatTensor)
    
    loss_ = criterion(pred, real) * mask 
    return torch.mean(loss_)

In [42]:
# Device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

## TODO: Combine the encoder and decoder into one class
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)
decoder = Decoder(vocab_tar_size, embedding_dim, units, units, BATCH_SIZE)

encoder.to(device)
decoder.to(device)

optimizer = optim.Adam(list(encoder.parameters()) + list(decoder.parameters()), 
                       lr=0.001)

In [43]:
EPOCHS = 10

for epoch in range(EPOCHS):
    start = time.time()
    
    encoder.train()
    decoder.train()
    
    total_loss = 0
    
    for (batch, (inp, targ, inp_len)) in enumerate(dataset):
        loss = 0
        
        xs, ys, lens = sort_batch(inp, targ, inp_len)
        enc_output, enc_hidden = encoder(xs.to(device), lens, device)
        dec_hidden = enc_hidden
        
        # use teacher forcing - feeding the target as the next input (via dec_input)
        dec_input = torch.tensor([[targ_lang.word2idx['<start>']]] * BATCH_SIZE)
        
        # run code below for every timestep in the ys batch
        for t in range(1, ys.size(1)):
            predictions, dec_hidden, _ = decoder(dec_input.to(device), 
                                         dec_hidden.to(device), 
                                         enc_output.to(device))
            loss += loss_function(ys[:, t].to(device), predictions.to(device))
            #loss += loss_
            dec_input = ys[:, t].unsqueeze(1)
            
        
        batch_loss = (loss / int(ys.size(1)))
        total_loss += batch_loss
        
        optimizer.zero_grad()
        
        loss.backward()

        ### UPDATE MODEL PARAMETERS
        optimizer.step()
        
        if batch % 100 == 0:
            print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1,
                                                         batch,
                                                         batch_loss.detach().item()))
        
        
    ### TODO: Save checkpoint for model
    print('Epoch {} Loss {:.4f}'.format(epoch + 1,
                                        total_loss / N_BATCH))
    print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))
    
    
    
    

Epoch 1 Batch 0 Loss 4.5204
Epoch 1 Batch 100 Loss 1.6360
Epoch 1 Batch 200 Loss 1.4364
Epoch 1 Batch 300 Loss 1.3506
Epoch 1 Loss 1.5437
Time taken for 1 epoch 163.88275265693665 sec

Epoch 2 Batch 0 Loss 0.9812
Epoch 2 Batch 100 Loss 0.7181
Epoch 2 Batch 200 Loss 0.7679
Epoch 2 Batch 300 Loss 0.7228
Epoch 2 Loss 0.8083
Time taken for 1 epoch 160.36644506454468 sec

Epoch 3 Batch 0 Loss 0.5852
Epoch 3 Batch 100 Loss 0.6267
Epoch 3 Batch 200 Loss 0.5714
Epoch 3 Batch 300 Loss 0.4395
Epoch 3 Loss 0.4494
Time taken for 1 epoch 161.67758560180664 sec

Epoch 4 Batch 0 Loss 0.3149
Epoch 4 Batch 100 Loss 0.2192
Epoch 4 Batch 200 Loss 0.2611
Epoch 4 Batch 300 Loss 0.2966
Epoch 4 Loss 0.2576
Time taken for 1 epoch 161.3397660255432 sec

Epoch 5 Batch 0 Loss 0.1515
Epoch 5 Batch 100 Loss 0.1362
Epoch 5 Batch 200 Loss 0.1718
Epoch 5 Batch 300 Loss 0.1483
Epoch 5 Loss 0.1604
Time taken for 1 epoch 161.23252367973328 sec

Epoch 6 Batch 0 Loss 0.0931
Epoch 6 Batch 100 Loss 0.0576
Epoch 6 Batch 200 